In [10]:
import numpy as np
import pandas as pd

In [11]:
df = pd.read_csv('pesajes-01-01-2019-31-12-2020_anonimizado.csv' )
df['fecha'] = pd.to_datetime(df['fecha'])

df_precios = pd.read_csv('precios_predio.csv')
df = df.fillna('No especificado')

/home/val/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


fecha        datetime64[ns]
etapa                object
bolson               object
camion               object
peso                  int64
cartonero            object
legacyId             object
predio               object
material             object
dtype: object

In [12]:
def agregar_tipoCartonero(df):
    df_tipoCartonero = df.copy()
    df_tipoCartonero["tipoCartonero"] = "RA"
    indices_LE = df_tipoCartonero[df_tipoCartonero["legacyId"].str.startswith("LE")].index
    indices_none = df_tipoCartonero[df_tipoCartonero["legacyId"] == "No especificado"].index

    df_tipoCartonero.loc[indices_LE, "tipoCartonero"] = "LE"
    df_tipoCartonero.loc[indices_none, "tipoCartonero"] = "No especificado"

    return df_tipoCartonero

df = agregar_tipoCartonero(df)

In [13]:
def retornar_df_pagos(df_filtrado, df_precios):
    df_pagos = df_filtrado.copy()
    df_pagos["precio/kg"] = 0
    df_pagos["precio"] = 0
    for index, row in df_precios.iterrows():
        sede = row["Sede"]
        material = row["Material"]
        tipoCartonero = row["tipoCartonero"]
        indices = df_pagos[(df_pagos["predio"] == sede) & (df_pagos["material"] == material) & (df_pagos["tipoCartonero"] == tipoCartonero)].index

        precio = row["Precio"]
        df_pagos.loc[indices, "precio/kg"] = precio
        
    df_pagos["precio"] = df_pagos["precio/kg"]*df_pagos["peso"] 
    return df_pagos
        
df_pagos = retornar_df_pagos(df, df_precios)
df_pagos

,fecha,etapa,bolson,camion,peso,cartonero,legacyId,predio,material,tipoCartonero,precio/kg,precio
0,2019-12-02,R26,55.0,HVS 858,29,COMPA1987,3633,CORTEJARENA,No especificado,RA,0,0
1,2019-12-02,R26,28.0,HVS 858,93,COMPA687,3599,CORTEJARENA,No especificado,RA,0,0
2,2019-12-02,R26,61.0,HVS 858,35,COMPA2144,3638,CORTEJARENA,No especificado,RA,0,0
3,2019-12-02,R26,33.0,HVS 858,27,COMPA972,3608,CORTEJARENA,No especificado,RA,0,0
4,2019-12-02,R26,30.0,HVS 858,45,COMPA693,3603,CORTEJARENA,No especificado,RA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
89945,2020-12-30,E13,92.0,No especificado,57,COMPA669,5526,SAAVEDRA,Mezcla B,RA,25,1425
89946,2020-12-30,E13,152.0,No especificado,76,COMPA1240,4972,SAAVEDRA,Mezcla B,RA,25,1900
89947,2020-12-30,E13,224.0,No especificado,33,COMPA597,6376,SAAVEDRA,Mezcla B,RA,25,825
89948,2020-12-30,E13,224.0,No especificado,46,COMPA597,6376,SAAVEDRA,Mezcla B,RA,25,1150


In [14]:
def pago_por_compa(df_filtrado, df_precios):
    df_pagos = retornar_df_pagos(df, df_precios)
    df_pagos = df_pagos.groupby(['cartonero']).sum('precio')
    return df_pagos[['precio']]

pago_por_compa(df, df_precios)

,precio
cartonero,
COMPA0,160
COMPA1,15240
COMPA10,5305
COMPA100,0
COMPA1000,11320
...,...
COMPA995,2205
COMPA996,4660
COMPA997,36720


In [15]:
def pago_por_predio(df, df_precios):
    df_pagos = retornar_df_pagos(df, df_precios)
    df_pagos = df_pagos.groupby('predio').sum('precio')
    return df_pagos[['precio']]

pago_por_predio(df, df_precios)

,precio
predio,
AVELLANEDA,199965
BARRACAS,17130320
CORTEJARENA,0
No especificado,0
SAAVEDRA,14272650


In [16]:
def pago_total(df, df_precios):
    df_pagos = retornar_df_pagos(df, df_precios)
    df_pagos = df_pagos['precio'].sum()
    return df_pagos

pago_total(df, df_precios)

31602935